### FRA

+ `admin_name1` = `region`
+ `admin_name2` = `city`
+ `place_name` = `city`

In [392]:
import pandas as pd
import warnings
from unidecode import unidecode

warnings.filterwarnings("ignore")

**Crunchbase Dataframe**

In [393]:
df = pd.read_csv('input/foodtech.csv')
df = df[df['country_code'] == 'FRA']
df = df[['uuid','country_code', 'state_code', 'region', 'city', 'address', 'postal_code']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5532 entries, 218 to 163804
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uuid          5532 non-null   object
 1   country_code  5532 non-null   object
 2   state_code    0 non-null      object
 3   region        5532 non-null   object
 4   city          5532 non-null   object
 5   address       3575 non-null   object
 6   postal_code   3589 non-null   object
dtypes: object(7)
memory usage: 345.8+ KB


*Reformatting*

In [394]:
df['region'] = df['region'].str.replace(r'\bPoitou-Charentes\b', 'Aquitaine')


In [395]:
df['city'] = df['city'].apply(unidecode)
df['region'] = df['region'].apply(unidecode)

df['city'] = df['city'].str.lower()
df['region'] = df['region'].str.lower()



**Postal Codes**

In [396]:
codes = pd.read_json('input/geonames.json')
codes = codes[codes['country_code'] == 'FR']

In [397]:
codes['admin_name1'] = codes['admin_name1'].apply(lambda x: unidecode(x) if x is not None else None)
codes['admin_name2'] = codes['admin_name2'].apply(lambda x: unidecode(x) if x is not None else None)
codes['place_name'] = codes['place_name'].apply(lambda x: unidecode(x) if x is not None else None)

In [398]:
codes.loc[(codes['admin_name1'] == 'Grand Est') & (codes['admin_name2'] == 'Bas-Rhin'), 'admin_name1'] = 'Alsace'
codes.loc[codes['admin_name2'] == 'Haut-Rhin', 'admin_name1'] = 'Alsace'

codes.loc[codes['admin_name2'] == 'Doubs', 'admin_name1'] = 'Franche-Comte'
codes.loc[codes['admin_name2'] == 'Oise', 'admin_name1'] = 'Picardie'
codes.loc[codes['admin_name2'] == 'Haute-Loire', 'admin_name1'] = 'Auvergne'
codes.loc[codes['admin_name2'] == 'Aude', 'admin_name1'] = 'Languedoc-Roussillon'
codes.loc[codes['admin_name2'] == 'Herault', 'admin_name1'] = 'Languedoc-Roussillon'

codes['admin_name1'] = codes['admin_name1'].str.replace(r'\bNouvelle-Aquitaine\b', 'Aquitaine')
codes['admin_name1'] = codes['admin_name1'].str.replace(r'\bAuvergne-Rhone-Alpes\b', 'Rhone-Alpes')
codes['admin_name1'] = codes['admin_name1'].str.replace(r'\bOccitanie\b', 'Midi-Pyrenees')
codes['admin_name1'] = codes['admin_name1'].str.replace(r'\bCentre-Val de Loire\b', 'Centre')
codes['admin_name1'] = codes['admin_name1'].str.replace(r'\bNormandie\b', 'Basse-Normandie')
codes['admin_name1'] = codes['admin_name1'].str.replace(r'\bGrand Est\b', 'Champagne-Ardenne')
codes['admin_name1'] = codes['admin_name1'].str.replace(r'\bHauts-de-France\b', 'nord-pas-de-calais')
codes['admin_name1'] = codes['admin_name1'].str.replace(r'\bBourgogne-Franche-Comte\b', 'Bourgogne')



`admin_name2`

In [399]:
codes1 = codes.copy()
codes1 = codes1[['postal_code','admin_name1', 'admin_name2']]

In [400]:
codes1['admin_name1'] = codes1['admin_name1'].str.lower()
codes1['admin_name2'] = codes1['admin_name2'].str.lower()

codes1.drop_duplicates(subset=['admin_name1', 'admin_name2'], inplace=True)

In [401]:
merged_df = pd.merge(df, codes1, left_on=['city', 'region'], right_on=['admin_name2', 'admin_name1'], how='left')
merged_df.drop_duplicates(subset=['uuid'], inplace=True)

In [402]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5532 entries, 0 to 5531
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           5532 non-null   object
 1   country_code   5532 non-null   object
 2   state_code     0 non-null      object
 3   region         5532 non-null   object
 4   city           5532 non-null   object
 5   address        3575 non-null   object
 6   postal_code_x  3589 non-null   object
 7   postal_code_y  820 non-null    object
 8   admin_name1    820 non-null    object
 9   admin_name2    820 non-null    object
dtypes: object(10)
memory usage: 475.4+ KB


`place_name`

In [403]:
codes2 = codes.copy()
codes2 = codes2[['postal_code','admin_name1', 'place_name']]

codes2['admin_name1'] = codes2['admin_name1'].str.lower()
codes2['place_name'] = codes2['place_name'].str.lower()

codes2.drop_duplicates(subset=['admin_name1', 'place_name'], inplace=True)


In [404]:
merged_df = pd.merge(merged_df, codes2, left_on=['city', 'region'], right_on=['place_name', 'admin_name1'], how='left')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5532 entries, 0 to 5531
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           5532 non-null   object
 1   country_code   5532 non-null   object
 2   state_code     0 non-null      object
 3   region         5532 non-null   object
 4   city           5532 non-null   object
 5   address        3575 non-null   object
 6   postal_code_x  3589 non-null   object
 7   postal_code_y  820 non-null    object
 8   admin_name1_x  820 non-null    object
 9   admin_name2    820 non-null    object
 10  postal_code    4742 non-null   object
 11  admin_name1_y  4742 non-null   object
 12  place_name     4742 non-null   object
dtypes: object(13)
memory usage: 605.1+ KB


**Filling NaNs with Existing Postal Codes**

In [405]:
merged_df['postal_code_y'].fillna(merged_df['postal_code'], inplace=True)
merged_df['postal_code_y'].fillna(merged_df['postal_code_x'], inplace=True)

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5532 entries, 0 to 5531
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           5532 non-null   object
 1   country_code   5532 non-null   object
 2   state_code     0 non-null      object
 3   region         5532 non-null   object
 4   city           5532 non-null   object
 5   address        3575 non-null   object
 6   postal_code_x  3589 non-null   object
 7   postal_code_y  5302 non-null   object
 8   admin_name1_x  820 non-null    object
 9   admin_name2    820 non-null    object
 10  postal_code    4742 non-null   object
 11  admin_name1_y  4742 non-null   object
 12  place_name     4742 non-null   object
dtypes: object(13)
memory usage: 605.1+ KB


*Across rows: df*

In [406]:
merged_df = merged_df[merged_df.columns[:-5]]

In [407]:
codes_df = df[df['postal_code'].notna()]

codes_df = codes_df[['region', 'city', 'postal_code']]
codes_df.drop_duplicates(subset=['region', 'city'], inplace=True)

In [408]:
merged_df = pd.merge(merged_df, codes_df, left_on=['city', 'region'], right_on=['city', 'region'], how='left')
merged_df['postal_code_y'].fillna(merged_df['postal_code'], inplace=True)

In [409]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5532 entries, 0 to 5531
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           5532 non-null   object
 1   country_code   5532 non-null   object
 2   state_code     0 non-null      object
 3   region         5532 non-null   object
 4   city           5532 non-null   object
 5   address        3575 non-null   object
 6   postal_code_x  3589 non-null   object
 7   postal_code_y  5398 non-null   object
 8   postal_code    4815 non-null   object
dtypes: object(9)
memory usage: 432.2+ KB


*Checking NaNs*

In [410]:
nan_df = merged_df[merged_df['postal_code_y'].isna()]
nan_df['region'].value_counts()

languedoc-roussillon          14
auvergne                      13
lorraine                      13
picardie                      12
limousin                      11
aquitaine                     10
rhone-alpes                    9
franche-comte                  7
centre                         7
haute-normandie                7
bourgogne                      5
pays de la loire               5
bretagne                       4
midi-pyrenees                  4
provence-alpes-cote d'azur     3
ile-de-france                  3
champagne-ardenne              2
nord-pas-de-calais             2
basse-normandie                1
alsace                         1
corse                          1
Name: region, dtype: int64

*Save to csv*

In [411]:
merged_df = merged_df.rename(columns={'postal_code_x': 'pc_crunchbase', 
                                      'postal_code_y': 'pc_filled'})

merged_df = merged_df[['uuid', 'pc_crunchbase', 'pc_filled']]


In [412]:
merged_df.to_csv('processed/FRA_processed.csv')